In [1]:
import requests
from IPython.display import clear_output
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.poolmanager import PoolManager
import ssl

In [24]:
def baixar_imagem(url, nome_arquivo):
    """
    Baixa uma imagem a partir de uma URL e a salva localmente com o nome especificado.

    Parâmetros:
    url (str): A URL da imagem que será baixada.
    nome_arquivo (str): O nome do arquivo onde a imagem será salva.

    Funcionalidade:
    - Faz uma requisição GET para a URL fornecida.
    - Se a requisição for bem-sucedida (status code 200), a imagem é salva no caminho especificado.
    - Caso contrário, imprime uma mensagem de erro com o status code.

    Exemplo:
    >>> baixar_imagem('https://example.com/image.jpg', 'imagem_local.jpg')
    Imagem baixada com sucesso como imagem_local.jpg
    """
    response = requests.get(url)
    if response.status_code == 200:
        with open(nome_arquivo, 'wb') as arquivo:
            arquivo.write(response.content)
            print(f"Imagem baixada com sucesso como {nome_arquivo}")
    else:
        print(f"Falha ao baixar a imagem: Status {response.status_code}\nImagem erro em: {nome_arquivo}")


In [25]:
import pandas as pd 
def load_excel_as_dict(file_excel,arrColumns=['cimagem','links']):
    """
    Lê um arquivo Excel e retorna um DataFrame contendo apenas as colunas especificadas.

    Parâmetros:
    pathExcel (str): O caminho para o arquivo Excel que será lido.
    arrColumns (list): Lista com os nomes das colunas que devem ser carregadas do arquivo Excel.

    Retorna:
    pd.DataFrame: Um DataFrame contendo apenas as colunas especificadas pelo parâmetro `arrColumns`.

    Exemplo:
    >>> df = lerExcel('dados.xlsx', ['cimagem', 'links'])
    >>> print(df)
       Nome  Idade
    0  Ana     30
    1  João    25
    """
    pathfile = '../Fonte/excel/' + file_excel
    
    df = pd.read_excel(pathfile, usecols=arrColumns)
    return df

In [26]:
def load_text_as_dict_columns(file_txt,file_type='default_value'):
    """
    Lê um arquivo de texto e retorna uma lista de strings, correspondentes ao conteúdo das linhas do arquivo.

    Parâmetros:
    file_txt (str): O nome do arquivo de texto que será lido.
    file_type (str): O tipo de dados que o arquivo contém. Deve ser 'cimagem' ou 'links'. 
                     Se outro valor for fornecido, uma exceção será levantada.

    Retorna:
    list: Uma lista de strings contendo o conteúdo das linhas do arquivo.

    Funcionalidade:
    - Verifica se o `file_type` é 'cimagem' ou 'links'.
    - Lê o arquivo de texto especificado.
    - Converte cada linha do arquivo em uma string, removendo espaços em branco no início e no fim.
    - Salva as strings em um dicionário com a chave `file_type`.
    - Retorna a lista de strings correspondente ao `file_type`.

    Exceções:
    - Levanta um ValueError se `file_type` não for 'cimagem' ou 'links'.
    - Imprime uma mensagem e não levanta exceção se o arquivo não for encontrado.

    Exemplo:
    >>> load_text_as_dict_columns('codigos.txt', 'cimagem')
    ['12345', '67890', '24680']
    """
    if file_type not in ['cimagem', 'links']:
        print('file_type deve ser cimagem ou links')
        raise ValueError("file_type deve ser 'cimagem' ou 'links'")
    
    pathfile = '../Fonte/txt/' + file_txt
    Dados = { file_type : []}
    try:    
        with open(pathfile, 'r') as arquivo:
            linhas = arquivo.readlines()
            # Converter a lista de strings em uma lista de inteiros
            Dados[file_type] = [str(linha.strip()) for linha in linhas]
            print(f'Arquivo {file_txt} salvo no Dict Dados[{file_type}]')
    except FileNotFoundError:
        print(f'Arquivo: {file_txt} não encontrado.')
    return Dados[file_type]

In [27]:
def load_txt_in_dict(file_column1,file_column2):
    """
    Lê dois arquivos de texto e retorna um dicionário com listas de strings, correspondentes ao conteúdo das linhas dos arquivos.

    Parâmetros:
    file_column1 (str): O nome do primeiro arquivo de texto, que será lido como 'cimagem'.
    file_column2 (str): O nome do segundo arquivo de texto, que será lido como 'links'.

    Retorna:
    dict: Um dicionário contendo duas chaves:
          'cimagem' - uma lista de strings correspondente ao conteúdo das linhas de `file_column1`.
          'links' - uma lista de strings correspondente ao conteúdo das linhas de `file_column2`.

    Funcionalidade:
    - Lê `file_column1` e salva seu conteúdo em uma lista associada à chave 'cimagem'.
    - Lê `file_column2` e salva seu conteúdo em uma lista associada à chave 'links'.
    - Retorna um dicionário com as duas listas.

    Exemplo:
    >>> load_txt_in_dict('codigos.txt', 'download_links.txt')
    {'cimagem': ['12345', '67890', '24680'], 'links': ['http://example.com/img1.jpg', 'http://example.com/img2.jpg','http://example.com/img3.jpg']}
    """
    Dados = { 'cimagem' : [],'links' : []}
    Dados['cimagem'] = load_text_as_dict_columns(file_column1,'cimagem')
    Dados['links'] = load_text_as_dict_columns(file_column2,'links')
    return Dados

In [2]:
class SSLAdapter(HTTPAdapter):
    def init_poolmanager(self, *args, **kwargs):
        context = ssl.create_default_context()
        context.set_ciphers("DEFAULT:@SECLEVEL=1")  # Reduz o nível de segurança para aceitar chaves menores
        kwargs['ssl_context'] = context
        return super(SSLAdapter, self).init_poolmanager(*args, **kwargs)

    def proxy_manager_for(self, *args, **kwargs):
        context = ssl.create_default_context()
        context.set_ciphers("DEFAULT:@SECLEVEL=1")
        kwargs['ssl_context'] = context
        return super(SSLAdapter, self).proxy_manager_for(*args, **kwargs)

In [3]:
# Criando uma sessão com o adaptador SSL personalizado
session = requests.Session()
session.mount('https://', SSLAdapter())

In [4]:
def baixar(fonte='excel'):
    match fonte:
        case 'excel':
            Dados = load_excel_as_dict('Dados.xlsx')
        case 'txt':
            Dados = load_txt_in_dict('codigos.txt','download_links.txt')
        case _:
            print('Nada foi carregado.\nCarregue: EXCEL ou TXT')
        
    if len(Dados['cimagem']) != len(Dados['links']):
        print('Arrays com tamanho diferentes')
        raise Exception('Arrays com tamanho diferentes')
    else:
        None      
    return Dados

In [5]:
Dados = baixar('excel')

NameError: name 'load_excel_as_dict' is not defined

In [31]:
init = 0
total = len(Dados['cimagem'])

for index, cimagem in enumerate(Dados['cimagem'][init:], start=init):
    file_txt = f'../Images/Entrada/{cimagem}'
    link = Dados["links"][index]
    baixar_imagem(link, file_txt)
    
    clear_output(wait=False)
    print(f'Progresso : {index + 1}/{total} downloads')
    

SSLError: HTTPSConnectionPool(host='www.vonder.com.br', port=443): Max retries exceeded with url: /estatico/vonder/produto/3599470180.jpg (Caused by SSLError(SSLError(1, '[SSL: DH_KEY_TOO_SMALL] dh key too small (_ssl.c:1006)')))